In [1]:
# Подключаем модуль случайных чисел 
import random 
# Подключаем модуль для Телеграма 
import telebot 

from typing import List, Dict

from natasha import (Segmenter,
                     NewsEmbedding,
                     NewsMorphTagger,
                     NewsSyntaxParser,
                     Doc)

from natasha import MorphVocab
# Указываем токен 
bot = telebot.TeleBot('1477804768:AAFEsN415MIVXlmNVrKIBN4eCwZJhVBRXuw') 
# Импортируем типы из модуля, чтобы создавать кнопки 
from telebot import types 

from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
from numpy.random import choice
import wikipediaapi
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup

wiki_wiki = wikipediaapi.Wikipedia('ru', extract_format=wikipediaapi.ExtractFormat.HTML)

owm = OWM('12a5d3d53aa91916d14df3bd70ae5d33')

In [2]:
wiki_pages = {'Дитя погоды':'фильм', 'Твое имя':'фильм',
              'Макото Синкай':"человека по имени", 'Хаяо Миядзаки': "человека по имени",
             ' RADWIMPS': 'группу', 'Пять сантиметров в секунду': 'фильм'}

In [3]:
def get_random_fact():
    random_page = choice(list(wiki_pages.keys()))
    page_py = wiki_wiki.page(random_page)
    soup = BeautifulSoup(page_py.text, 'html.parser')
    spans = soup.findAll("p")
    return f"Интересный факт про {wiki_pages[random_page]} {random_page}.\n" + choice(list(spans)).text

In [4]:
class Weather:
    CLOUD = {0: 'солнечно ☀️', 1: "небольшая облачность 🌤", 2: "облачность 🌥", 3: "пасмурно ☁️"}
    WIND = {0: 'штиль', 1: 'легкий ветер', 2: 'довольно ветренно', 3: 'сильный ветер', 4: 'буря'}
    _WIND = {0: 3, 1: 5, 2: 10 ,3: 17}
    DATE = {0: 'Сегодня', 1: "Завтра", 2: "Послезавтра"}
    CITY = {'спб': 'Saint Petersburg, RU', 'мск': 'Moscow, RU'}
    def __init__(self, temp, feels_like, wind, date, clouds, snow, rain):
        self.temp = int(temp)
        self.feels_like = int(feels_like)
        self.wind = wind
        self.date = date
        self.clouds = clouds
        self.snow = snow
        self.rain = rain
        
    def __str__(self):
        msg = f"{self.DATE[self.date]} будет {self.WIND[self.wind]} и {self.CLOUD[self.clouds]}, температура {self.temp}°C"
        if (self.feels_like != None):
            msg += f", ощущается как {self.feels_like}°C"
        rain_snow = []
        if (self.snow):
            rain_snow.append("снег ❄️")
        if (self.rain):
            rain_snow.append("дождь ☔️")
        if len(rain_snow) > 0:
            msg += ". Ожидается " + " и ".join(rain_snow)
        msg += "."
        return msg
    
    @staticmethod
    def get_weather(city: str, date):
        mgr = owm.weather_manager()
        observation = mgr.weather_at_place(Weather.CITY[city])
        if date == 0:
            w = observation.weather
        elif date == 1:
            forecaster = mgr.forecast_at_place(Weather.CITY[city], '3h')
            tomorrow = timestamps.tomorrow()                                   
            w = forecaster.get_weather_at(tomorrow) 
        else:
            forecaster = mgr.forecast_at_place(Weather.CITY[city], '3h')
            now = timestamps.now()
            two_days_date = now.date() + timedelta(days=2)
            tomorrow = datetime(two_days_date.year, two_days_date.month, two_days_date.day,
                    now.hour, now.minute, 0, 0, timezone.utc)                              
            w = forecaster.get_weather_at(tomorrow) 
            
        temp = w.temperature('celsius')['temp']
        feels_like = w.temperature('celsius')['feels_like']
        clouds = min([int(w.clouds/25), 3])
        wind = Weather._get_wind(w.wind()['speed'])
        snow = (len(w.snow) > 0)
        rain = (len(w.rain) > 0)
        return Weather(temp, feels_like, wind, date, clouds, snow, rain)
        
    @staticmethod
    def _get_wind(speed: float):
        for i in Weather._WIND.keys():
            if speed < Weather._WIND[i]:
                return i
        return 4
        
    @staticmethod
    def random(date):
        return Weather(random.randint(-30, 30), random.randint(0, 3), random.randint(0, 6), date)
        

In [5]:
class MsgParser:
    def __init__(self, text):
        self.segmenter = Segmenter()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)
        self.doc = Doc(text)
        self.doc.segment(self.segmenter)
        self.doc.tag_morph(self.morph_tagger)
        self.doc.parse_syntax(self.syntax_parser)
        self.morph_vocab = MorphVocab()
        for token in self.doc.tokens:
            token.lemmatize(self.morph_vocab)
        
        
    @staticmethod
    def get_cities(list_tokens:list, word_forms: dict) -> List[str]:
        """
        Return list of cities from sentence.
        """
        CITIES = ['мск', "спб"]
        AVAILABLE_CITIES = {'мск': 'мск', "москва": "мск", "спб": "спб",
                            "питер": "спб", "санкт-петербург": "спб",
                            "петербург": "спб", 'ленинград': "спб"}
        cities = []
        for token in list_tokens:
            if ("Case" in token.feats) and (token.feats['Case'] == 'Loc'):
                cities.append(word_forms[token.text])
            else:
                if (word_forms[token.text] in CITIES):
                    cities.append(word_forms[token.text])

        filtered = []
        for city in cities:
            try:
                filtered.append(AVAILABLE_CITIES[city])
            except KeyError:
                pass
        return list(set(filtered))
    
    def check_token(self, word: str) -> bool:
        for tok in self.doc.tokens:
            if (tok.lemma == word) or (tok.text == word):
                return True
        return False
        

In [6]:
last_msg = None
# Метод, который получает сообщения и обрабатывает их 
@bot.message_handler(content_types=['text']) 
def get_text_messages(message):
    last_msg = message
    handle_msg(message)
    
def handle_msg(message):    
    goodbye_flag = False
    no_msg_flag = True
    
    # Если написали «Привет»
    if message.text == "/help": 
        bot.send_message(message.from_user.id, "Привет, меня зовут Хина!"+ 
                         " Я умею предсказывать погоду и немного управлять ей."+
                         " Могу рассказать тебе про погоду на ближайшие пару дней :)")
    
    else:
        parser = MsgParser(message.text)
        if parser.check_token("погода") or parser.check_token("давать") or parser.check_token("давать"):
            no_msg_flag = False
            cities = []
            lemmatized = {_.text : _.lemma for _ in parser.doc.tokens}
            for sent in parser.doc.sents:
                cities.extend(MsgParser.get_cities(sent.morph.tokens, lemmatized))
            cities = list(set(cities))
            
            day = 0
            if parser.check_token("завтра"):
                day = 1
            elif parser.check_token("послезавтра"):
                day = 2

            # если город не указан
            if len(cities) == 0:
                bot.send_message(message.from_user.id, "Привет! Погоду в каком городе ты хочешь узнать?") 
                # Готовим кнопки 
                keyboard = types.InlineKeyboardMarkup() 
                # По очереди готовим текст и обработчик для каждого знака зодиака 
                key_msk = types.InlineKeyboardButton(text='Москва', callback_data='city_msk') 
                # И добавляем кнопку на экран 
                keyboard.add(key_msk) 

                key_spb = types.InlineKeyboardButton(text='Питер', callback_data='city_spb') 
                keyboard.add(key_spb) 
                bot.send_message(message.from_user.id, text='Выбери город', reply_markup=keyboard)
            else:
                if 'спб' in cities:
                    bot.send_message(message.from_user.id, "Погода в Питере.\n" + str(Weather.get_weather('спб', day)))
                if 'мск' in cities:
                    bot.send_message(message.from_user.id, "Погода в Москве.\n" + str(Weather.get_weather("мск", day)))
                    
        elif parser.check_token("привет"):
            no_msg_flag = False
            # Пишем приветствие 
            bot.send_message(message.from_user.id, "Привет, меня зовут Хина!"+ 
                         " Я умею предсказывать погоду и немного управлять ей."+
                         " Могу рассказать тебе про погоду на ближайшие пару дней :)")
        elif parser.check_token("картинка"):
            no_msg_flag = False
            bot.send_photo(message.from_user.id, photo=open(f'bot_pic/{random.randint(1, 10)}.jpg', 'rb'))
        elif parser.check_token("факт") or parser.check_token("интерес") or parser.check_token("интересный"):
            no_msg_flag = False
            # Пишем приветствие 
            bot.send_message(message.from_user.id, get_random_fact())
        if parser.check_token("спасибо"): 
            no_msg_flag = False
            # Пишем приветствие 
            bot.send_message(message.from_user.id, "Не за что :)")
        if parser.check_token("пока"):
            no_msg_flag = False
            bot.send_message(message.from_user.id, "Пока, приятно было пообщаться :)")
            goodbye_flag = True
        if no_msg_flag: 
            bot.send_message(message.from_user.id, "Я не понимаю тебя :( Попробуй ввести /help")
            
        if not goodbye_flag:
            bot.send_message(message.from_user.id, "Я могу еще чем-то помочь?")

In [7]:
# Обработчик нажатий на кнопки 
@bot.callback_query_handler(func=lambda call: True) 
def callback_worker(call):
    # Если нажали на одну из 12 кнопок — выводим гороскоп 
    if call.data == "city_msk": 
        # Формируем гороскоп 
        bot.send_message(call.message.chat.id, str(Weather.get_weather('мск', 0)))
    elif call.data == "city_spb": 
        # Формируем гороскоп 
        bot.send_message(call.message.chat.id, str(Weather.get_weather('спб', 0)))
    

In [ ]:
# Запускаем постоянный опрос бота в Телеграме 
while(True):
    try:
        bot.polling(none_stop=True, interval=0)
    except Exception:
        # пытаемся еще раз обработать сообщение
        try:
            handle_msg(last_msg)
        except Exception:
            pass